# Library

In [1]:
from module_KK import * 

# Auto reload of library
%reload_ext autoreload
%autoreload 2

# Data Preprocessing

In [7]:
# df = preprocessing_KTX()
df = pd.read_csv(os.path.join('.', 'Data', 'df_KTX_KK.csv')).iloc[:,1:]
df

,전체주중주말,주운행선,운행년월,일수,주말수,주중수,공휴일수,명절수,공급차량수,공급좌석합계수,...,운행대비고객이동,관광,일반,일반/관광,대수송,임시,확정,시발역,종착역,열차운행횟수
0,전체,경부선,2015-01-01,31,14,17,1,0,69372,3543395,...,47.537163,0.0,475.0,2.0,0.0,10.0,467.0,186,186,4035
1,전체,경부선,2015-02-01,28,12,16,3,3,63270,3245162,...,42.343794,0.0,432.0,3.0,91.0,12.0,332.0,168,168,3700
2,전체,경부선,2015-03-01,31,13,18,1,0,68492,3508931,...,50.965278,0.0,465.0,4.0,0.0,0.0,469.0,186,186,3999
3,전체,경부선,2015-04-01,30,12,18,0,0,66146,3397973,...,47.501813,0.0,479.0,9.0,0.0,0.0,488.0,180,180,3810
4,전체,경부선,2015-05-01,31,15,16,3,0,69858,3592956,...,46.144244,0.0,515.0,10.0,0.0,21.0,504.0,186,186,4022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,주중,호남선,2023-11-01,18,0,18,0,0,12816,664148,...,35.003224,0.0,41.0,490.0,0.0,0.0,531.0,126,126,972
1412,주중,호남선,2023-12-01,16,0,16,1,0,11408,591470,...,30.390615,0.0,44.0,434.0,0.0,2.0,476.0,112,112,866
1413,주중,호남선,2024-01-01,19,0,19,1,0,13544,705158,...,38.587825,0.0,37.0,527.0,0.0,2.0,562.0,133,133,1028
1414,주중,호남선,2024-02-01,17,0,17,1,1,12176,633908,...,29.497230,0.0,86.0,412.0,55.0,2.0,441.0,119,119,922


In [ ]:
# date_to_stat 함수
# 데이터분리
# 스케일링
# 모델링